In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import re

In [ ]:
df = pd.read_csv('cheapcarco.csv')
df

In [ ]:
df.info()

"""
While checking the CSV file, I found some empty rows and removed them manually.
One car, "2000 Toyota Estima Silver Wagon", has no transmission type, so I set it to None.
"""

In [ ]:
"""drop duplicates"""
df.drop_duplicates()

""" change price to int"""
df['price'] = (df['price'].str.replace(r'[\$,]','',regex=True)).astype(int)

"""add column year made"""
df['year_made'] = df['name'].str[:4].astype(int)

""" add columm car brand"""
pattern = re.compile(r'^\d{4}\s+([a-zA-z]+)')
df['brand']=df['name'].str.extract(pattern, expand = False)

""" add column car name"""
pattern = re.compile(r'^\s*\d{4}\s+\S+\s+(.*)\s*\.?$')
df ['name'] = df['name'].str.extract(pattern, expand = False)

""" change odometer to int"""
df['odometer'] = (df['odometer'].str.replace(r'[\s[Kms,]','',regex=True)).astype(int)

""" change cylinder to int"""
df['cylinder'] = (df['cylinder'].str.replace(r'[Cylinders]+','',regex=True)).astype(int)
